In [1]:
# !git clone https://github.com/fadhleryani/malti_arabi_fst.git
# # !git pull

# %pip install pynini
# %pip install pyfoma

In [2]:
# %cd malti_arabi_fst

In [1]:
import pynini as pn
import kenlm
from itertools import product


In [2]:
malti2arabi = pn.string_file('malti2arabi.map')


malti_sigma = pn.project(malti2arabi,'input')
arabi_sigma = pn.project(malti2arabi,'output') 


sigma_input = pn.union(malti_sigma ).closure()

transcriber = pn.union(malti2arabi ).closure()



dediac_cross = pn.string_file('dediac.map')

diacs = 'ًٌٍَُِّْ'
dediac = pn.cdrewrite(dediac_cross,'','',pn.union(arabi_sigma,*diacs).closure())

words = pn.string_file('tn-maghreb-words.txt') @ dediac


In [3]:
# test dediac
('ابَطا' @ dediac).string()

'ابطا'

In [4]:
def get_paths(fst):
    return list(fst.paths().items())


# get_paths('merħba' @ rwrs @ transcriber)



fst = ('<BOS>malta<EOS>' )  @ transcriber @ dediac @ words

get_paths(fst)

[('<BOS>malta<EOS>', 'ملت', <tropical Weight 0 at 0x10c0445b0>),
 ('<BOS>malta<EOS>', 'ملط', <tropical Weight 0 at 0x10c0444f0>),
 ('<BOS>malta<EOS>', 'مالت', <tropical Weight 0 at 0x10c0445f0>),
 ('<BOS>malta<EOS>', 'مالطا', <tropical Weight 0 at 0x10c0446d0>),
 ('<BOS>malta<EOS>', 'مالطا', <tropical Weight 0 at 0x10c044530>)]

In [60]:
[][:0]

[]

In [69]:
def translit_fst(tok):
    fst = (f'<BOS>{tok}<EOS>')  @ transcriber @ dediac @ pn.cdrewrite(pn.cross('+ ',''),'','',arabi_sigma.closure()) 
    return fst

def translit_sentence(sentence,limit_backoff_options=3):
    lattice = []
    for tok in sentence.split():
        options_fst = translit_fst(tok) @ words
        if not get_paths(options_fst): # backoff
            options_fst = translit_fst(tok)            
            options = [x[1] for x in get_paths(options_fst)][:limit_backoff_options]
            if limit_backoff_options==0:
                options = ['']
            elif limit_backoff_options <0:
                raise Exception('limit_backoff_options must be larger than zero')
        else:
            options = [x[1] for x in get_paths(options_fst)]
        lattice.append(options)
    # return lattice
    return list({' '.join(x) for x in list(product(*lattice))})

translit_sentence('min feyn')

['من فين', 'من فاين', 'مين فين', 'مين فاين']

In [47]:
fst = (f'<BOS>feyn<EOS>')  @ transcriber  @ pn.cdrewrite(pn.cross('+ ',''),'','',arabi_sigma.closure()) @ dediac
get_paths(fst)

[('<BOS>feyn<EOS>', 'فين', <tropical Weight 0 at 0x10d2ddc50>),
 ('<BOS>feyn<EOS>', 'فاين', <tropical Weight 0 at 0x10d2dda90>)]

In [39]:
model = kenlm.Model('aggregated_country/lm/word/tn-maghreb.arpa')

Loading the LM will be faster if you build a binary file.
Reading /Users/f/ba3sasah/malti_arabi_fst/aggregated_country/lm/word/tn-maghreb.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [70]:
def score_generated_sentences(sentences):
    return sorted([(x,model.score(x)) for x in sentences],key=lambda y: -y[1])


score_generated_sentences(translit_sentence('malta uffiċjalment',limit_backoff_options=1))


[('مالطا ففتشيلمنت', -13.266424179077148),
 ('مالت ففتشيلمنت', -13.84896183013916),
 ('ملت ففتشيلمنت', -13.84896183013916),
 ('ملط ففتشيلمنت', -14.07813835144043)]